<a href="https://colab.research.google.com/github/wannasmile/colab_code_note/blob/main/BI0006.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime, os

from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

import tensorflow as tf
from tensorflow import keras

import warnings
warnings.filterwarnings("ignore")

In [2]:
!wget https://raw.githubusercontent.com/wannasmile/colab_code_note/main/ChurnModelling.csv

--2024-03-04 17:28:58--  https://raw.githubusercontent.com/wannasmile/colab_code_note/main/ChurnModelling.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 684858 (669K) [text/plain]
Saving to: ‘ChurnModelling.csv.1’

ChurnModelling.csv. 100%[===================>] 668.81K  4.10MB/s    in 0.2s    

2024-03-04 17:28:59 (4.10 MB/s) - ‘ChurnModelling.csv.1’ saved [684858/684858]



say goodbye to pandas documentation

In [3]:
dataset = pd.read_csv("ChurnModelling.csv") # 加载数据

In [4]:
!pip install duckdb

In [5]:
dataset.head(10)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
5,6,15574012,Chu,645,Spain,Male,44,8,113755.78,2,1,0,149756.71,1
6,7,15592531,Bartlett,822,France,Male,50,7,0.00,2,1,1,10062.80,0
7,8,15656148,Obinna,376,Germany,Female,29,4,115046.74,4,1,0,119346.88,1
8,9,15792365,He,501,France,Male,44,4,142051.07,2,0,1,74940.50,0
9,10,15592389,H?,684,France,Male,27,2,134603.88,1,1,1,71725.73,0


In [6]:
import duckdb
data_groupby_gender = duckdb.query("select Gender, sum(Exited) as ExitedCnt from dataset group by Gender").df()
data_groupby_gender.head()

,Gender,ExitedCnt
0,Male,898.0
1,Female,1139.0


In [7]:
import duckdb
dataset = shuffle(dataset)
data_balance_groupby_cid = duckdb.query('''

    select t.*
    from
    (
    select CustomerId
    ,Geography
    ,Gender
    ,Age
    ,Balance
    ,avg(Balance) over (partition by Geography order by Balance asc ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) as GeographyBalanceAvg
    ,avg(Balance) over (partition by Gender order by Balance asc ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) as GenderBalanceAvg
    ,avg(Balance) over (partition by Age order by Balance asc ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) as AgeBalanceAvg
    ,row_number() over (order by RowNumber asc) as rn
    from dataset
    where Geography='France'
    --limit 10
    ) t
    where rn<=10

    UNION ALL

    select t.*
    from
    (
    select CustomerId
    ,Geography
    ,Gender
    ,Age
    ,Balance
    ,avg(Balance) over (partition by Geography order by Balance asc ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) as GeographyBalanceAvg
    ,avg(Balance) over (partition by Gender order by Balance asc ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) as GenderBalanceAvg
    ,avg(Balance) over (partition by Age order by Balance asc ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) as AgeBalanceAvg
    ,row_number() over (order by RowNumber asc) as rn
    from dataset
    where Geography='Spain'
    --limit 10
    ) t
    where rn<=10

    UNION ALL

    select t.*
    from
    (
    select CustomerId
    ,Geography
    ,Gender
    ,Age
    ,Balance
    ,avg(Balance) over (partition by Geography order by Balance asc ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) as GeographyBalanceAvg
    ,avg(Balance) over (partition by Gender order by Balance asc ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) as GenderBalanceAvg
    ,avg(Balance) over (partition by Age order by Balance asc ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) as AgeBalanceAvg
    ,row_number() over (order by RowNumber asc) as rn
    from dataset
    where Geography='Germany'
    --limit 10
    ) t
    where rn<=10
''').df()
data_balance_groupby_cid.head(30)

,CustomerId,Geography,Gender,Age,Balance,GeographyBalanceAvg,GenderBalanceAvg,AgeBalanceAvg,rn
0,15634602,France,Female,42,0.00,62092.636516,60322.670159,63533.193831,1
1,15619304,France,Female,42,159660.80,62092.636516,60322.670159,63533.193831,2
2,15701354,France,Female,39,0.00,62092.636516,60322.670159,57124.620149,3
3,15592531,France,Male,50,0.00,62092.636516,63546.284875,69868.251034,4
4,15792365,France,Male,44,142051.07,62092.636516,63546.284875,67810.752927,5
5,15592389,France,Male,27,134603.88,62092.636516,63546.284875,57378.123190,6
6,15767821,France,Male,31,102016.72,62092.636516,63546.284875,62892.045505,7
7,15632264,France,Female,34,0.00,62092.636516,60322.670159,59226.748826,8
8,15691483,France,Female,25,0.00,62092.636516,60322.670159,54215.392609,9
9,15568982,France,Female,24,0.00,62092.636516,60322.670159,57562.646712,10


In [8]:
import duckdb
dataset = shuffle(dataset)
data_balance_groupby_cid = duckdb.query('''

    select t.*
    from
    (
    select CustomerId
    ,Balance
    ,avg(Balance) over (partition by Geography order by Balance asc ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) as GeographyBalanceAvg
    ,avg(Balance) over (partition by Gender order by Balance asc ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) as GenderBalanceAvg
    ,avg(Balance) over (partition by Age order by Balance asc ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) as AgeBalanceAvg
    ,row_number() over (order by RowNumber asc) as rn
    from dataset
    where Geography='France'
    --limit 10
    ) t
    where rn<=10

    UNION ALL

    select t.*
    from
    (
    select CustomerId
    ,Balance
    ,avg(Balance) over (partition by Geography order by Balance asc ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) as GeographyBalanceAvg
    ,avg(Balance) over (partition by Gender order by Balance asc ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) as GenderBalanceAvg
    ,avg(Balance) over (partition by Age order by Balance asc ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) as AgeBalanceAvg
    ,row_number() over (order by RowNumber asc) as rn
    from dataset
    where Geography='Spain'
    --limit 10
    ) t
    where rn<=10

    UNION ALL

    select t.*
    from
    (
    select CustomerId
    ,Balance
    ,avg(Balance) over (partition by Geography order by Balance asc ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) as GeographyBalanceAvg
    ,avg(Balance) over (partition by Gender order by Balance asc ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) as GenderBalanceAvg
    ,avg(Balance) over (partition by Age order by Balance asc ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) as AgeBalanceAvg
    ,row_number() over (order by RowNumber asc) as rn
    from dataset
    where Geography='Germany'
    --limit 10
    ) t
    where rn<=10
''').df()
data_balance_groupby_cid.head(30)

,CustomerId,Balance,GeographyBalanceAvg,GenderBalanceAvg,AgeBalanceAvg,rn
0,15634602,0.00,62092.636516,60322.670159,63533.193831,1
1,15619304,159660.80,62092.636516,60322.670159,63533.193831,2
2,15701354,0.00,62092.636516,60322.670159,57124.620149,3
3,15592531,0.00,62092.636516,63546.284875,69868.251034,4
4,15792365,142051.07,62092.636516,63546.284875,67810.752927,5
5,15592389,134603.88,62092.636516,63546.284875,57378.123190,6
6,15767821,102016.72,62092.636516,63546.284875,62892.045505,7
7,15632264,0.00,62092.636516,60322.670159,59226.748826,8
8,15691483,0.00,62092.636516,60322.670159,54215.392609,9
9,15568982,0.00,62092.636516,60322.670159,57562.646712,10
